### Set up an environment

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE COMPUTE_WH;

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE COMPUTE_WH;

-- create a database and a schema for raw data
CREATE DATABASE IF NOT EXISTS RAW_DATA;
CREATE SCHEMA IF NOT EXISTS RAW_DATA.LICHESS;

-- create a table for data
CREATE TABLE IF NOT EXISTS RAW_DATA.LICHESS.MY_GAMES (DATA VARIANT);

-- create temporary table for JSON files
CREATE TABLE IF NOT EXISTS RAW_DATA.LICHESS.MY_GAMES_STAGING (JSON_STR VARIANT);
--TRUNCATE TABLE RAW_DATA.LICHESS.MY_GAMES_STAGING;

In [ ]:
USE DATABASE RAW_DATA;
USE SCHEMA LICHESS;

### Create a file format

In [ ]:
CREATE OR REPLACE FILE FORMAT RAW_DATA.LICHESS.LICHESS_JSON_GZ
  TYPE = JSON
  COMPRESSION = GZIP;


CREATE OR REPLACE STAGE raw_data.lichess.stg_lichess
    FILE_FORMAT = (FORMAT_NAME = lichess_json_gz_format);

### Check uploaded files

In [ ]:
LIST @stg_lichess

### Check data without loading

In [ ]:
SELECT TYPEOF($1) AS t, $1
FROM @stg_lichess (FILE_FORMAT => 'RAW_DATA.LICHESS.LICHESS_JSON_GZ')
LIMIT 5;

### Copy data from Stage to a Table

In [ ]:
select * from @stg_lichess;
--  internal stage is populated

In [ ]:
COPY INTO RAW_DATA.LICHESS.MY_GAMES_STAGING FROM @raw_data.lichess.stg_lichess file_format= (format_name = 'raw_data.lichess.lichess_json_gz')
FORCE = TRUE;

In [ ]:
select * from raw_data.lichess.my_games_staging;

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA()").show()


In [ ]:
df = session.table("RAW_DATA.LICHESS.MY_GAMES_STAGING")
df.show()

### Create View from JSON

In [ ]:
select distinct(JSON_str:winner::STRING) from RAW_DATA.LICHESS.MY_GAMES_STAGING

In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW
    view_lichess_data
AS SELECT

    DATE(JSON_STR:createdAt::INTEGER) as date_of_game,
    JSON_STR:moves::STRING as moves,

    JSON_STR:opening.eco::STRING as eco,
    SPLIT_PART(JSON_STR:opening.name::STRING, ':', 1) as opening_family,
    JSON_STR:opening.name::STRING as opening_name,
    JSON_STR:opening.ply::INTEGER as ply,

    JSON_STR:perf::STRING as time_control, 

    
    JSON_STR:players.white.rating::INTEGER as white_rating,
    JSON_STR:players.white.ratingDiff::INTEGER as white_rating_diff,
    JSON_STR:players.white.user.name::STRING as white_name,

    JSON_STR:players.black.rating::INTEGER as black_rating,
    JSON_STR:players.black.ratingDiff::INTEGER as black_rating_diff,
    JSON_STR:players.black.user.name::STRING as black_name,

    JSON_STR:rated::STRING as rated,
    JSON_STR:status::STRING as status,
    JSON_str:variant::STRING as game_variant,
    JSON_str:winner::STRING as winner,

    (CASE
        when JSON_STR:players.white.user.name::STRING = 'arek313' and JSON_str:winner::STRING = 'white' then 1
        when JSON_STR:players.white.user.name::STRING = 'arek313' and JSON_str:winner::STRING = 'black' then 0
        when JSON_STR:players.white.user.name::STRING = 'arek313' and JSON_str:winner::STRING ='' then 0.5
        when JSON_STR:players.black.user.name::STRING = 'arek313' and JSON_str:winner::STRING = 'white' then 0
        when JSON_STR:players.black.user.name::STRING = 'arek313' and JSON_str:winner::STRING = 'black' then 1
        when JSON_STR:players.black.user.name::STRING = 'arek313' and JSON_str:winner::STRING ='' then 0.5
        else 0.5
    END)::FLOAT AS score,

    (CASE
        when JSON_STR:players.white.user.name::STRING = 'arek313' then 'white'
        when JSON_STR:players.black.user.name::STRING = 'arek313' then 'black'
    END)::STRING AS color

FROM RAW_DATA.LICHESS.MY_GAMES_STAGING

WHERE time_control in ('bullet', 'classic', 'blitz', 'rapid') --check how classical is defined. I don't have any in this period of time,
-- and white_name = 'arek313' or black_name = 'arek313 ';
    
    

In [ ]:
-- CREATE OR REPLACE VIEW
--     view_lichess_data_arek313_black
-- AS SELECT

--     DATE(JSON_STR:createdAt::INTEGER) as date_of_game,
--     JSON_STR:moves::STRING as moves,

--     JSON_STR:opening.eco::STRING as eco,
--     SPLIT_PART(JSON_STR:opening.name::STRING, ':', 1) as opening_family,
--     JSON_STR:opening.name::STRING as opening_name,
--     JSON_STR:opening.ply::INTEGER as ply,

--     JSON_STR:perf::STRING as time_control, 

    
--     JSON_STR:players.white.rating::INTEGER as white_rating,
--     JSON_STR:players.white.ratingDiff::INTEGER as white_rating_diff,
--     JSON_STR:players.white.user.name::STRING as white_name,

--     JSON_STR:players.black.rating::INTEGER as black_rating,
--     JSON_STR:players.black.ratingDiff::INTEGER as black_rating_diff,
--     JSON_STR:players.black.user.name::STRING as black_name,

--     JSON_STR:rated::STRING as rated,
--     JSON_STR:status::STRING as status,
--     JSON_str:variant::STRING as game_variant,
--     JSON_str:winner::STRING as winner,

--     (CASE
--         when JSON_str:winner::STRING = 'white' then 1
--         when JSON_str:winner::STRING = 'black' then 0
--         else 0.5
--     END)::FLOAT AS score

-- FROM RAW_DATA.LICHESS.MY_GAMES_STAGING

-- WHERE time_control in ('bullet', 'classic', 'blitz', 'rapid') --check how classical is defined. I don't have any in this period of time,
-- and black_name = 'arek313';
    
    

In [ ]:
select * from view_lichess_data_arek313_white
limit 5;

### Convert view to pandas dataframe

In [ ]:
df_view_lichess_data_arek313_white = session.table("view_lichess_data_arek313_white").to_pandas()

### Simple statistics

Check https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-visualize-data

In [ ]:
import matplotlib.pyplot as plt
import squarify

### How many games you won

In [ ]:
counts = df_view_lichess_data_arek313_white["SCORE"].value_counts(normalize=True).mul(100)
counts.plot(kind="bar")
plt.ylabel("%")
plt.title("Udział wartości w kolumnie 'status'")
plt.show()
plt.show()

In [ ]:
select * from view_lichess_data_arek313_black
where score = 0
limit 5;

In [ ]:
select 
    (SUM(CASE WHEN score= 1 THEN 1 ELSE 0 END) / COUNT(*)) AS win,
    (SUM(CASE WHEN score = 0.5 THEN 1 ELSE 0 END) / COUNT(*)) AS draw,
    (SUM(CASE WHEN score = 0 THEN 1 ELSE 0 END) / COUNT(*)) AS lose
from view_lichess_data_arek313_black

In [ ]:
plt.figure(figsize=())
squarify.plot(sizes=counts.values)

In [ ]:
-- df_view_lichess_data_arek313_white.groupby('score').'sum

In [ ]:
select count(*) as amount_of_games, sum(score)/count(*) as score, avg(white_rating), avg(ply), opening_family, opening_name
from view_lichess_data_arek313_white
where date_of_game > '2025-01-01'
group by opening_family, opening_name
having count(*) >= 3
order by score desc;

#### Score - strategiczna mapa debiutów 

In [ ]:
select count(*) as amount_of_games, sum(score)/count(*) as score, opening_family
from view_lichess_data_arek313_black
--where date_of_game > '2025-01-01'
group by opening_family
having count(*) >= 5
--order by score desc;

### How it looks like when it comes to 'ply'

In [ ]:
select * from view_lichess_data_arek313_black
where opening_family = 'Caro-Kann Defense'

#### Games in date

In [ ]:
select count(*), LEFT(date_of_game, 7) as month from view_lichess_data_arek313_black
group by 2
order by 2;

In [ ]:
SELECT AVG(white_rating), count(*), opening_name from view_lichess_data
WHERE black_name = 'arek313'
group by 3
having count(*) > 2
order by 1 desc;

In [ ]:
SELECT AVG(black_rating) from view_lichess_data
WHERE white_name = 'arek313';